# 🥱 RAG 篇_評估效能_快速入門 RAGAS

❤️ Created by [hengshiousheu](https://huggingface.co/Heng666).

#使用 Langchain 與 RAGAS 來進行 RAG 管道評估

歡迎使用 ragas 快速入門。我們將讓您盡快啟動並執行 ragas，以便您在這個程式庫確保您的變更改善您的整個管道時，可以回去改善您的檢索擴增產生管道。

在以下筆記本中，我們將討論 LLM Ops 的主要組成部分：

我們今天將利用 RAGAS 框架進行評估，因為它正成為評估 (至少是方向性) RAG 系統的標準方法。

我們還將討論一些更強大的檢索系統，這些系統有可能提高我們世代的質量！

讓我們先開始安裝套件包建置環境吧！



---



#快速入門篇

##環境建置

In [ ]:
!pip install -U -q langchain openai ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.8/735.8 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


Ragas 也能使用 OpenAI 來執行一些量測指標，因此請確保您的 OpenAI 金鑰已準備好並在您的環境中可用

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## 預備資料

Ragas 會對您的 RAG 管線執行 `ground_truth` 免費評估。這是因為對大多數人來說，建立一個高品質標記的資料集，以表示他們在生產中獲得的分配，是一個非常昂貴的過程。

因此，要使用 Ragas，您只需要以下資料
- quesiont: `list[str]` - RAG 管線將會評估的問題。
- answer: `list[str]` - RAG 管線產生的答案並提供給使用者。
- ocntexts: `list[list[str]]` - 傳入 LLM 以回答問題的內容。

在此，我們使用我們 [Financial Opinion Mining and Question Answering (fiqa) Dataset](https://sites.google.com/view/fiqa/) 資料集 建立的其中一個基線範例資料集。


In [ ]:
## 讀取 HuggingFace Hub 上資料
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

FileNotFoundError: Couldn't find a dataset script at /content/explodinggradients/fiqa/fiqa.py or any data file in the same directory. Couldn't find 'explodinggradients/fiqa' on the Hugging Face Hub either: FileNotFoundError: Dataset 'explodinggradients/fiqa' doesn't exist on the Hub. If the repo is private or gated, make sure to log in with `huggingface-cli login`.

## 使用指標

Ragas 會根據兩個維度來衡量您的管線效能

1. Faithfulness：衡量產生的答案與給定內容的事實一致性。
2. Relevancy：衡量檢索到的內容和產生的答案與問題有多相關

現在讓我們匯入這些指標，並進一步了解它們表示的含義

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

在這裡看到我們使用了 4 個指標，但它們代表什麼？

1. answer_relevancy - 衡量答案與問題有多相關

2. faithfulness - 答案對基於問題內容的真實一致性。

3. context_recall: 衡量檢索器檢索回答問題所需所有必要資訊的能力。

4. context_precision - 衡量檢索到的內容與問題有多相關。 傳達檢索管道的品質。



**請留意:** *預設情況下，這些指標使用 OpenAI 的 API 來計算分數。如果您使用此指標，請務必使用您的 API 金鑰設定環境金鑰 `OPENAI_API_KEY` 。您也可以嘗試其他 LLMs 進行評估，查看 llm 指南以了解更多資訊*



## 開始評估

執行評估就像在 Dataset 上呼叫評估，並使用您選擇的指標一樣簡單。

In [ ]:
from ragas import evaluate

result = evaluate(
    fiqa_eval["baseline"].select(range(1)),
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall
    ],
)

result

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9825, 'context_recall': 0.5000}

接著你就有上述所需的指標，其中 `ragas_score` 為您提供單一指標，供您使用，而其他指標則測量您的管線的不同部分。

現在，將取得的結果轉換為 pandas 陣列，來檢視結果。

In [ ]:
df = result.to_pandas()
df.head()

,question,ground_truths,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,Have the check reissued to the proper payee.Ju...,1.0,1.0,0.982533,0.5


如此一來你就達成了，初探 RAGAS 使用公開財務金融資料集來進行 RAG 評估。

#導入 LangChain 進行評估

在以下筆記本中，我們將討論 LLM Ops 的主要組成部分：
- 評估

接著將繼續使用 RAGAS 框架進行評估，這有機會成為 RAG 系統環節中的標準方法。
同時，我們也在測試不同 Retrieval 情況下，提高系統的評分方式。

讓我們像往常開始一樣，先建置我們所需的環境吧！

## 環境建置

In [ ]:
!pip install -U -q langchain openai ragas arxiv pymupdf chromadb wandb tiktoken tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key: ··········


## 收集資料

今天將使用 Arxiv 的論文作為我們的資料。
建立 RAG 框架，來研讀論文

我們可以使用 Langchain 當中的 `ArxivLoader`輕鬆的讀取文件
這次我們抓最多 10 份文件。

- 點我查看[`ArxivLoader`](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.arxiv.ArxivLoader.html)

In [ ]:
from langchain.document_loaders import ArxivLoader

base_docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=5).load()
len(base_docs)

5

In [ ]:
for doc in base_docs:
  print(doc.metadata)

{'Published': '2022-02-13', 'Title': 'A Survey on Retrieval-Augmented Text Generation', 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu', 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention\nof the computational linguistics community. Compared with conventional\ngeneration models, retrieval-augmented text generation has remarkable\nadvantages and particularly has achieved state-of-the-art performance in many\nNLP tasks. This paper aims to conduct a survey about retrieval-augmented text\ngeneration. It firstly highlights the generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable approaches according to different tasks\nincluding dialogue response generation, machine translation, and other\ngeneration tasks. Finally, it points out some important directions on top of\nrecent methods to facilitate future research.'}
{'Published': '2023-12-09', 'Title': 'Context Tuning for Retrieval Augmented Generation', 'Autho

## RAG 索引階段 - 建立 Index

讓我們使用一個簡單的索引建立策略，僅在我們的文件中使用 RecursiveCharacterTextSplitter ，並使用 OpenAIEmbeddings() 將每個文件嵌入到我們的 VectorStore 中。

- 點我查看[`RecursiveCharacterTextSplitter()`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=250)
docs = text_splitter.split_documents(base_docs)
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## RAG 檢索階段 - 檢索相近文件

讓我們使用 .as_retriever() 方法將我們的 Chroma 向量儲存轉換為檢索器。
並且讓 Retriever 找回 Chroma DB 當中相關的 5 筆資料回來。

In [ ]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k" : 5})

讓我們來開始問問題

In [ ]:
relevant_docs = base_retriever.get_relevant_documents("What is Retrieval Augmented Generation?")

In [ ]:
for rel_doc in relevant_docs:
  print(rel_doc)

page_content='et al., 2021b), and knowledge-intensive generation\n(Lewis et al., 2020b). Finally, we also point out\nsome promising directions on retrieval-augmented\ngeneration to push forward the future research.\n2\nRetrieval-Augmented Paradigm' metadata={'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu', 'Published': '2022-02-13', 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention\nof the computational linguistics community. Compared with conventional\ngeneration models, retrieval-augmented text generation has remarkable\nadvantages and particularly has achieved state-of-the-art performance in many\nNLP tasks. This paper aims to conduct a survey about retrieval-augmented text\ngeneration. It firstly highlights the generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable approaches according to different tasks\nincluding dialogue response generation, machine translation, and other\ngeneration tasks. Finall

### RAG 生成階段 - 建立檢索增強用 Prompt

現在，我們可以設定一個提示範本，用於為LLM提供必要的背景、使用者查詢和說明！

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

### 建立基本 RAG Pipe

現在我們可以建構我們基本 RAG Chain！

In [ ]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question")
    | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


讓我們測試一下

In [ ]:
question = "What is RAG?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result)

{'response': AIMessage(content='Answer: RAG stands for Retrieval Augmented Generation.'), 'context': [Document(page_content='2020). RAG consists of three primary components:\nTool Retrieval, Plan Generation, and Execution.1\nIn this study, we focus on enhancing tool retrieval,\nwith the goal of achieving subsequent improve-\nments in plan generation.', metadata={'Authors': 'Raviteja Anantha, Tharun Bethi, Danil Vodianik, Srinivas Chappidi', 'Published': '2023-12-09', 'Summary': "Large language models (LLMs) have the remarkable ability to solve new tasks\nwith just a few examples, but they need access to the right tools. Retrieval\nAugmented Generation (RAG) addresses this problem by retrieving a list of\nrelevant tools for a given task. However, RAG's tool retrieval step requires\nall the required information to be explicitly present in the query. This is a\nlimitation, as semantic search, the widely adopted tool retrieval method, can\nfail when the query is incomplete or lacks context

## 建立評估用資料集

這環節我們將使用 GPT-3.5-turbo 來建立資料集

可能會需要花費場時間來建立

基本流程是，使用 LangChain 根據我們問的問題，以及找到的內容來建立答案

讓我們來看看如何在程式碼當中如何實現。

使用 Langchain output_parsers 來建立輸出格式

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [ ]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"question": string  // a question about the context.\n}\n```'

#### 利用 GPT-3.5-turbo 來進行提問，藉此建立 Question

In [ ]:
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

因為讀取的為論文，因此讓 LLM 先作為大學教授來考學生

In [ ]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=docs[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

來看一下考題如何

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
What is the main objective of the paper 'A Survey on Retrieval-Augmented Text Generation'?
context
{'page_content': 'A Survey on Retrieval-Augmented Text Generation\nHuayang Li♥,∗\nYixuan Su♠,∗\nDeng Cai♦,∗\nYan Wang♣,∗\nLemao Liu♣,∗\n♥Nara Institute of Science and Technology\n♠University of Cambridge\n♦The Chinese University of Hong Kong\n♣Tencent AI Lab', 'metadata': {'Published': '2022-02-13', 'Title': 'A Survey on Retrieval-Augmented Text Generation', 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu', 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention of the computational linguistics community. Compared with conventional generation models, retrieval-augmented text generation has remarkable advantages and particularly has achieved state-of-the-art performance in many NLP tasks. This paper aims to conduct a survey about retrieval-augmented text generation. It firstly highlights the generic paradigm of retrieval-augmented 

看起來不錯，接著讓我們來產生大量考題吧，放入 List 當中

In [ ]:
from tqdm import tqdm

qac_triples = []

for text in tqdm(docs[:10]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 10/10 [00:21<00:00,  2.16s/it]


讓我們看一下內容

In [ ]:
qac_triples[2]

{'question': "What is the aim of the paper 'A Survey on Retrieval-Augmented Text Generation'?",
 'context': Document(page_content='Yan Wang♣,∗\nLemao Liu♣,∗\n♥Nara Institute of Science and Technology\n♠University of Cambridge\n♦The Chinese University of Hong Kong\n♣Tencent AI Lab\nli.huayang.lh6@is.naist.jp, ys484@cam.ac.uk\nthisisjcykcd@gmail.com, brandenwang@tencent.com', metadata={'Published': '2022-02-13', 'Title': 'A Survey on Retrieval-Augmented Text Generation', 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu', 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention\nof the computational linguistics community. Compared with conventional\ngeneration models, retrieval-augmented text generation has remarkable\nadvantages and particularly has achieved state-of-the-art performance in many\nNLP tasks. This paper aims to conduct a survey about retrieval-augmented text\ngeneration. It firstly highlights the generic paradigm of retrieval-

#### 利用 GPT-4 來回答問題，藉此建立 Ground_truth

In [ ]:
answer_generation_llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

讓我們查看一下

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

answer
The purpose of this paper is to conduct a comprehensive survey on the topic of retrieval-augmented text generation. It aims to highlight the generic paradigm of retrieval-augmented generation models, review notable approaches within various NLP tasks such as dialogue response generation and machine translation, and identify important future research directions in this field.
question
What is the purpose of this paper?


大量建立答案

In [ ]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████| 10/10 [01:18<00:00,  7.90s/it]


將剛剛建立的問題、答案都放入 `eval_dataset`當中

In [ ]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'context', 'metadata', 'ground_truth'],
    num_rows: 10
})

In [ ]:
eval_dataset.to_csv("groundtruth_eval_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

10076

#### (可選)如果不想自己建立資料集的話，也可以參考我建立的版本

In [ ]:
import requests

url = 'https://gist.githubusercontent.com/Heng-xiu/3c8d8fab601aa8f8ea9a98a43564f82b/raw/9a87a7ccc8d3d3e010ba12de52c868f92a21bc7d/groundtruth_eval_dataset.csv'  # 將 GIST_RAW_URL 替換成你的 Gist Raw URL
r = requests.get(url)
with open('groundtruth_eval_dataset.csv', 'wb') as file:
    file.write(r.content)


In [ ]:
from datasets import Dataset
eval_dataset = Dataset.from_csv("groundtruth_eval_dataset.csv")

Generating train split: 0 examples [00:00, ? examples/s]

## 使用 RAGAS 進行評估

現在讓我們開始吧，設定上非常直覺簡單，只要建立資料集，當中包含 `question`, `answer`, `context`, `ground_truths`，就可以開始使用框架進行評估。

導入要用到的指標

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate

建立 RAGAS 所需資料集格式

In [ ]:
def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})  #這一行是靈魂，使用 RAGAS 當中的 LLM 來產生答案
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"].content,
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

建立 ragas 評估函數

In [ ]:
def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

#### 來建立資料集吧

In [ ]:
from tqdm import tqdm
import pandas as pd

basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)
basic_qa_ragas_dataset

100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 10
})

In [ ]:
basic_qa_ragas_dataset[0]

{'question': 'What is the purpose of this paper?',
 'answer': 'Answer: The purpose of this paper is to propose Corrective Retrieval Augmented Generation (CRAG) to improve the robustness of generation in retrieval-augmented generation models.',
 'contexts': ['across both short- and long-form generation tasks.\nTo facilitate others to reproduce our results, we will\npublish all source code later.\nIn summary, our contributions in this paper are\nthree-fold: 1) This paper studies the scenarios',
  'three-fold: 1) This paper studies the scenarios\nwhere the retriever returns inaccurate results and,\nto the best of our knowledge, makes the first\nattempt to design corrective strategies for RAG to\nimprove its robustness. 2) A plug-and-play method',
  'In summary, our contributions in this paper are\nthree-fold: 1) This paper studies the scenarios\nwhere the retriever returns inaccurate results and,\nto the best of our knowledge, makes the first\nattempt to design corrective strategies for R

保存結果下來

In [ ]:
basic_qa_ragas_dataset.to_csv("basic_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

20787

#### 開始評估
最後讓我們查看執行結果

In [ ]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

Evaluating:   0%|          | 0/70 [00:00<?, ?it/s]

轉換成 Pandas Dataframe 來查看細節

In [ ]:
df = basic_qa_result.to_pandas()
df.head()

,question,answer,contexts,ground_truths,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall,context_relevancy,answer_correctness,answer_similarity
0,What is the purpose of this paper?,Answer: The purpose of this paper is to propos...,[across both short- and long-form generation t...,[The purpose of this paper is to conduct a com...,The purpose of this paper is to conduct a comp...,0.588889,1.0,0.979752,1.0,0.038462,0.474685,0.898740
1,What is the main focus of this paper?,The main focus of this paper is on improving t...,"[2Microsoft Research, Redmond, USA\n3Beijing N...",[The main focus of this paper is to conduct a ...,The main focus of this paper is to conduct a c...,0.000000,NaN,0.999998,1.0,0.000000,0.472073,0.888293
2,What is the aim of the paper 'A Survey on Retr...,The aim of the paper 'A Survey on Retrieval-Au...,[paper aims to conduct a survey about retrieva...,[The aim of the paper 'A Survey on Retrieval-A...,The aim of the paper 'A Survey on Retrieval-Au...,1.000000,NaN,1.000000,1.0,0.111111,0.675832,0.989042
3,What is the main focus of the paper 'A Survey ...,The main focus of the paper 'A Survey on Retri...,[paper aims to conduct a survey about retrieva...,[The main focus of the paper 'A Survey on Retr...,The main focus of the paper 'A Survey on Retri...,0.950000,1.0,1.000000,1.0,0.111111,0.823282,0.985435
4,What is the focus of this paper?,The focus of this paper is on advocating for r...,[We conclude this paper with a roadmap to adva...,[The focus of this paper is on retrieval-augme...,The focus of this paper is on retrieval-augmen...,0.755556,1.0,1.000000,1.0,0.038462,0.476545,0.906244


如此一來我們就建立好基線了

### 改變檢索策略

現在，我們可以測試變更檢索器如何影響我們的 RAGAS 評估！

讓我們先把 QA_chain 建立成 function，採用 Factory 方式產生不同策略的 Retriever

In [ ]:
def create_qa_chain(retriever):
  primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
  created_qa_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )

  return created_qa_chain

#### 策略ㄧ、上層文件檢索器

這個檢索方法大概如下

1. 取得使用者問題
2. 使用 Dense Vector Retrieval 檢索 Child Documents
3. 若 Child Documents 的 Parent Documents 相同，就會合併。
4. 使用內存儲存體中的各自父文件替換子文件。
5. 使用父文件來擴充生成。

可以查看 [LangChain Parent Document Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever)知道更多資訊。
這個方法在知名的 RAG 框架 LLama-index 當中被稱作為 [Auto-Merging Retriever](https://docs.llamaindex.ai/en/latest/examples/retrievers/auto_merging_retriever.html#)

要是懶得看，也可以參考該 Youtube 內容
https://www.youtube.com/watch?v=wQEl0GGxPcM




In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

vectorstore = Chroma(collection_name="split_parents", embedding_function=OpenAIEmbeddings())
store = InMemoryStore()


In [ ]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
parent_document_retriever.add_documents(base_docs)

讓我們創建、測試，然後評估我們的新鏈！

In [ ]:
parent_document_retriever_qa_chain = create_qa_chain(parent_document_retriever)

In [ ]:
parent_document_retriever_qa_chain.invoke({"question" : "What is RAG?"})["response"].content

'Answer: Retrieval-augmented generation (RAG) is a practicable complement to large language models (LLMs) that relies heavily on the relevance of retrieved documents to improve the accuracy of generated texts.'

In [ ]:
pdr_qa_ragas_dataset = create_ragas_dataset(parent_document_retriever_qa_chain, eval_dataset)

100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


In [ ]:
pdr_qa_ragas_dataset.to_csv("pdr_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

53043

In [ ]:
pdr_qa_result = evaluate_ragas_dataset(pdr_qa_ragas_dataset)

Evaluating:   0%|          | 0/70 [00:00<?, ?it/s]

In [ ]:
pdr_qa_df = pdr_qa_result.to_pandas()
pdr_qa_df.head()

,question,answer,contexts,ground_truths,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall,context_relevancy,answer_correctness,answer_similarity
0,What is the purpose of this paper?,Answer: The purpose of this paper is to propos...,[2021) show that CRAG can significantly improv...,[The purpose of this paper is to conduct a com...,The purpose of this paper is to conduct a comp...,0.75,1.000000,0.916159,1.0,0.036145,0.477715,0.910860
1,What is the main focus of this paper?,Answer: The main focus of this paper is on Cor...,[A Survey on Retrieval-Augmented Text Generati...,[The main focus of this paper is to conduct a ...,The main focus of this paper is to conduct a c...,1.00,0.666667,0.999999,1.0,0.028090,0.524267,0.897067
2,What is the aim of the paper 'A Survey on Retr...,Answer: The aim of the paper 'A Survey on Retr...,[A Survey on Retrieval-Augmented Text Generati...,[The aim of the paper 'A Survey on Retrieval-A...,The aim of the paper 'A Survey on Retrieval-Au...,1.00,1.000000,0.999999,1.0,0.012987,0.790665,0.980843
3,What is the main focus of the paper 'A Survey ...,Answer: The main focus of the paper is to cond...,[A Survey on Retrieval-Augmented Text Generati...,[The main focus of the paper 'A Survey on Retr...,The main focus of the paper 'A Survey on Retri...,1.00,1.000000,0.873603,1.0,0.008130,0.841168,0.964673
4,What is the focus of this paper?,Answer: The focus of this paper is to improve ...,[A Survey on Retrieval-Augmented Text Generati...,[The focus of this paper is on retrieval-augme...,The focus of this paper is on retrieval-augmen...,1.00,0.500000,0.999999,1.0,0.064706,0.527016,0.908002


#### 策略二、聯合檢索法

讓我們來瞅瞅，這個方法怎麼做的
此法本身是結合不同檢索演算法優勢，將結果整合到一起。最常做的方法是採用稀疏檢索器(BM25) 結合密集檢索器(embedding similarity)。

- **稀疏檢索器**:善長根據關鍵字尋找相關文件。
- **密集檢索器**:擅長根據語義相似性尋找相關文件。

整體步驟如下：
1. 取得使用者問題
2. 使用多種檢索器
 - 使用 BM25 稀疏向量檢索檢索文件
 - 使用密集向量檢索方法檢索文件
3. 根據其權重使用倒數排名融合演算法將檢索到的文件收集並「融合」成單一排名清單。
4. 使用融合後檔案來增強我們生成答案


你可以在 [LangChain EnsembleRetrievler](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble) 查看

In [ ]:
!pip install -q -U rank_bm25

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=75)
docs = text_splitter.split_documents(base_docs)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embedding)
chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.75, 0.25])

In [ ]:
ensemble_retriever_qa_chain = create_qa_chain(ensemble_retriever)

In [ ]:
ensemble_retriever_qa_chain.invoke({"question" : "What is RAG?"})["response"].content

'RAG stands for Retrieval Augmented Generation.'

In [ ]:
ensemble_qa_ragas_dataset = create_ragas_dataset(ensemble_retriever_qa_chain, eval_dataset)

100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


In [ ]:
ensemble_qa_ragas_dataset.to_csv("ensemble_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

24794

In [ ]:
ensemble_qa_result = evaluate_ragas_dataset(ensemble_qa_ragas_dataset)

evaluating with [context_precision]


100%|██████████| 1/1 [00:09<00:00,  9.19s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:22<00:00, 22.46s/it]


evaluating with [answer_relevancy]


...

100%|██████████| 1/1 [00:02<00:00,  2.80s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:15<00:00, 15.88s/it]


evaluating with [answer_similarity]


100%|██████████| 1/1 [00:00<00:00,  1.42it/s]


In [ ]:
ensemble_qa_df = ensemble_qa_result.to_pandas()
ensemble_qa_df.head()

,question,answer,contexts,ground_truths,context_precision,faithfulness,answer_relevancy,context_recall,context_relevancy,answer_correctness,answer_similarity
0,What is the purpose of this paper?,The purpose of this paper is to improve the ze...,[dation of future work due to their outstandin...,[The purpose of this paper is to conduct a com...,0.000000,0.250000,1.0,1.000000,0.080645,0.214195,0.856778
1,What is the main focus of this paper?,The main focus of this paper is on improving t...,[example of completions of the prompt by diffe...,[The main focus of this paper is to conduct a ...,0.000000,0.333333,1.0,0.333333,0.033898,0.430286,0.863999
2,What is the aim of the paper 'A Survey on Retr...,The aim of the paper 'A Survey on Retrieval-Au...,[A Survey on Retrieval-Augmented Text Generati...,[The aim of the paper 'A Survey on Retrieval-A...,0.804167,1.000000,1.0,1.000000,0.019231,0.493087,0.972346
3,What is the main focus of the paper 'A Survey ...,The main focus of the paper 'A Survey on Retri...,[A Survey on Retrieval-Augmented Text Generati...,[The main focus of the paper 'A Survey on Retr...,0.804167,1.000000,1.0,0.800000,0.017857,0.580133,0.987201
4,What is the focus of this paper?,The focus of this paper is on enhancing tool r...,"[Tool Retrieval, Plan Generation, and Executio...",[The focus of this paper is on retrieval-augme...,1.000000,1.000000,1.0,0.666667,0.023256,0.468033,0.872130


### 整合結果來進行評估不同檢索策略下的差異

In [ ]:
def create_df_dict(pipeline_name, pipeline_items):
  df_dict = {"name" : pipeline_name}
  for name, score in pipeline_items:
    df_dict[name] = score
  return df_dict

In [ ]:
basic_rag_df_dict = create_df_dict("basic_rag", basic_qa_result.items())
pdr_rag_df_dict = create_df_dict("pdr_rag", pdr_qa_result.items())
ensemble_rag_df_dict = create_df_dict("ensemble_rag", ensemble_qa_result.items())

In [ ]:
results_df = pd.DataFrame([basic_rag_df_dict, pdr_rag_df_dict, ensemble_rag_df_dict])

In [ ]:
results_df

,name,context_precision,faithfulness,answer_relevancy,context_recall,context_relevancy,answer_correctness,answer_similarity
0,basic_rag,0.363750,0.433333,0.999566,0.96,0.036458,0.349999,0.914271
1,pdr_rag,0.900000,0.633333,0.997567,1.00,0.027299,0.412959,0.936126
2,ensemble_rag,0.660833,0.858333,0.999564,0.88,0.030071,0.471398,0.924236


## 反思

您可以從上述結果中得出什麼結論？

用您自己的話描述指標表達的內容。